# Learning Outcomes
1) Gentle into on CV (a subfield of AI)
2) Recap image as Numpy array
3) Splitting/merging channels
4) Cropping
5) Mathematical operations
6) image blending (add 2 images tgt)

In [1]:
import sys
assert sys.version_info >=(3,7)

import cv2 as cv 
import numpy as np
from util_func import show_img

# Recap on images as Numpy array
There are 2  primary types of images: **grayscale** and **color**.

Grayscale:
matrix(2D array)
(h,w)

Color:
3D array
(h,w,channels)

In [ ]:
# Create a grayscale image
img=np.zeros((2,4),dtype=np.uint8) # {[0,0,0,0][0,0,0,0]}
print(img)

In [ ]:
img_color=cv.cvtColor(img,cv.COLOR_GRAY2BGR) #(h,w,channel)->(2,4,3) #from img above 
print(img_color)

In [ ]:
img[0,1]=50
img[1,2]=150

print(img)

In [ ]:
#grayscale->color, stay grayscale,not color
img_color=cv.cvtColor(img,cv.COLOR_GRAY2BGR)
print(img_color) #black/white/gray(150,150,150/50,50,50)

# Access elements in array

In [ ]:
img=cv.imread("images/lena.jfif")

a=img[49,219,2] ##take even is slightly slower
b=img.item(49,219,2)
a==b

In [ ]:
%timeit a =img[49,219,2]
%timeit b =img.item(49,219,2)


# Numpy slicing

In [ ]:
# left slice

h,w=img.shape[:2] #get h,w
yc,xc=h//2,w//2 #get centre

#slicing
topleft=img[:yc,:xc] #0 until h//2

show_img("topleft",topleft)

In [ ]:
# 60x60 squared central region of the image

centre=img[yc-30:yc+30, xc-30:xc+30]

show_img("centre",centre)


# Create a white image

In [ ]:
#method 2

img=np.zeros((200,200))+255 #all 1 array (no need *255 ye ok)
img=np.uint8(img)

show_img("white",img)


# Exercise

In [ ]:
# extract region of interest

img=cv.imread("images/flower.jfif")

show_img("img",img)

# There are 3 ways to get the indices needed

In [ ]:
[i for i in dir(cv) if i.startswith("EVENT")]

In [ ]:
#1st way (to get the 4 parameters) (x1,y1,x2,y2)
#tick 4 points to get the parameters in array, later can call
img=cv.imread("images/flower.jfif")
img_copy=img.copy()


def rect_region(event,x,y,flags,params):
    if event==cv.EVENT_LBUTTONDOWN:
        print((x,y))
        cv.circle(img,(x,y),1,(0,0,255),-1)
        cv.imshow("img",img)
        
cv.imshow("img",img)
cv.setMouseCallback("img",rect_region)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
#get the some indices of the image wanted
#take img_copy de yi ge xiao part after knowing the x1,y1,x2,y2

flower=img_copy[41:120,89:173]

show_img("flower",flower)

In [ ]:
#2nd way (x,y,w,h)
#drag the area of interest
bbox=cv.selectROI("crop",img_copy) #can select which part wan to crop,drag the area


flower=img_copy[int(bbox[1]):int(bbox[1]+bbox[3]),
                int(bbox[0]):int(bbox[0]+bbox[2])]

show_img("flower",flower)
                    

In [ ]:
# 3rd way: paint app

In [ ]:
img_arr=np.zeros((30,30),dtype=np.uint8)

#assign the white region
img_arr[:10,10:20]=255
img_arr[10:20,:10]=255
img_arr[10:20,20:]=255
img_arr[20:,10:20]=255

img=np.tile(img_arr,(3,3)) #3,3 for repeating for horizontal and vertical respectively

show_img("pattern",img)

# image cropping
why?
1) Remove unwanted object
2) Separate the image into a $3 \times 3$grids. We move/adjust the camera in such a way that the object of interest lies on the gridlines or their intersections. As such, your image would look more aethetically appealing. This is known as rule of thirds.
3) One of the image augmentation methods for DL model training

In [ ]:
img=cv.imread("images/dog.jfif")
img_copy=img.copy()

h,w=img.shape[:2]
#User defined parameters
n_vertical_grids=4
n_horizontal_grids=4

#1st rectangle
M=int(h/n_vertical_grids)
N=int(w/n_horizontal_grids)

tiles=[]

#draw the h,w rectangle
for y in range(0,h,M):
    for x in range(0,w,N):
        #adjust
        x1=x+N
        y1=y+M
        
        if x1>w and y1>h:
            x1=w-1 #拉回来
            y1=h-1
            cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
            tile=img[y:h,x:w] #put in array later can easily call the indices wanted
            tiles.append(tile)
            
        elif y1>h:
            y1=h-1
            cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
            tile=img[y:h,x:x1]
            tiles.append(tile)
        
        elif x1>w:
            x1=w-1
            cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
            tile=img[y:y1,x:w]
            tiles.append(tile)
            
        else:
            cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
            tile=img[y:y1,x:x1]
            tiles.append(tile)

show_img("crop",img_copy)

In [ ]:
show_img("patch",tiles[9])

# Splitting and merging colour channels

In [ ]:
img=cv.imread("images/lena.jfif")

(b,g,r)=cv.split(img) #below can use b,g,r
img_merge=cv.merge((b,g,r))

#show img,img_merge (split then merge) are same
np.array_equal(img,img_merge)


In [ ]:
#split the figure into b,g,r

import matplotlib.pyplot as plt

fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(12,4),sharey=True)
fig.suptitle("Different color channels")

ax1.imshow(b,cmap=plt.cm.gray)
ax1.set(title="blue channel",xticks=[],yticks=[])
ax2.imshow(g,cmap=plt.cm.gray)
ax2.set(title="green channel",xticks=[],yticks=[])
ax3.imshow(r,cmap=plt.cm.gray)
ax3.set(title="red channel",xticks=[],yticks=[])

plt.tight_layout() #show in 3 figure in row
plt.show()

# display different channels in color images

In [ ]:
#make G,R to be 0 and B=255 to look B more brighter

img=cv.imread("images/dog.jfif")

channels=cv.split(img)

#this color varible will be window names
colors=("blue","green","red")

imgs=[] #grab b,g,r into this imgs

for i,ch in enumerate(channels):
    img_arr=np.zeros_like(img)
    img_arr[...,i]=ch
    imgs.append(img_arr)
    
for c,img in zip(colors,imgs):
    cv.imshow(c,img)
    
cv.waitKey(0)
cv.destroyAllWindows()

# Point operators

Elementary math operations:addition, substraction, multiplication, and division

1) alpha = x or //
2) beta = + or -
when alpha>1, contrast will increase.

when 0<alpha<1, contrast will decrease.

when beta>0, brightness will increase.

when beta<0, brightness will decrease.

In [ ]:
np.uint8(np.array([-2,0,259,300])) #-2 is 254

In [ ]:
# alpha > 1 (high contrast)
# 0 < alpha < 1 (low contrast)
# beta > 0 (brightness high)
# beta < 0 (brightness low)

def point_op(img,alpha,beta):
    """point operators of image.Arguments:
    1. source image
    2. multiplier
    3. constant"""
    img=img.astype(float)
    res=alpha*img+beta #the formula
    res=np.clip(res,0,255) #lower and upper limit
    return np.uint8(res)

In [ ]:
img=cv.imread("images/bridge.jfif")

#increase the brightness and contrast
transform=point_op(img,1.6,20)

cv.imshow("original",img)
show_img("transform",transform)

In [ ]:
darken=point_op(img,1,-80)
show_img("img",darken)

# Gamma Correction

Technique to adjust the brightness of image.

#dark area become brighter

gamma<1, make dark area brighter

gamma>1, make dark area darker.

In [ ]:
#gamma can be high or low to adjust the brightness of the image
gamma=1/2.2 

lookUpTable=np.empty((1,256),dtype=np.uint8) #lookuptable:1-155

for i in range(256):
    lookUpTable[0,i]=np.clip(pow(i/255.0,gamma)*255.0,0,255) #clip: to put the formula inside # the formula 

img=cv.imread("images/mountains_prop.jpg")
res=cv.LUT(img,lookUpTable)

cv.namedWindow("original",cv.WINDOW_NORMAL)
cv.imshow("original",img)
show_img("gamma correction",res,adjust=True)

# Image blending (add 2 images)
```
cv.addWeighted(img1,alpha,img2,1-alpha,beta)
```


In [ ]:
#merge 2 images

img=cv.imread("images/lena.jfif")
img2=cv.imread("images/coins.jfif")

#resize img2 to be same dimension as img
h,w=img.shape[:2]
img2=cv.resize(img2,(w,h)) #w 1st
alpha=0.7

res=cv.addWeighted(img,alpha,img2, 1-alpha,0) #alpha:0.7 (img 70% visible), 1-alpha=0.3 (img2 30% visible)

cv.imshow("lena",img)
cv.imshow("resized coin",img2)
show_img("image blending",res)


# Exercises:

In [4]:
#Q1
width = 450
height = 350

random_noise_color_image = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8) #3 color channels (Red, Green, and Blue)

random_grayscale_image = np.random.randint(0, 256, (height, width), dtype=np.uint8)

cv.imshow("Noise Color", random_noise_color_image)
cv.imshow("Noise Grayscale", random_grayscale_image)
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
#Q2
def crop_grid (img, num_horizontal_grid, num_vertical_grid, line_color):
    img=cv.imread(img)
    img_copy=img.copy()

    h,w=img.shape[:2]
    #User defined parameters
    n_vertical_grids=num_vertical_grid
    n_horizontal_grids=num_horizontal_grid

    #1st rectangle
    M=int(h/n_vertical_grids)
    N=int(w/n_horizontal_grids)

    tiles=[]

    #draw the h,w rectangle
    for y in range(0,h,M):
        for x in range(0,w,N):
            #adjust
            x1=x+N
            y1=y+M
        
            if x1>w and y1>h:
                x1=w-1 #pull back
                y1=h-1
                cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
                tile=img[y:h,x:w] #put in array later can easily call the indices wanted
                tiles.append(tile)
            
            elif y1>h:
                y1=h-1
                cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
                tile=img[y:h,x:x1]
                tiles.append(tile)
        
            elif x1>w:
                x1=w-1
                cv.rectangle(img_copy,(x,y),(x1,y1),(0,255,0),1) #start,end,color,brightness
                tile=img[y:y1,x:w]
                tiles.append(tile)
            
            else:
                cv.rectangle(img_copy,(x,y),(x1,y1),line_color,1) #start,end,color,brightness
                tile=img[y:y1,x:x1]
                tiles.append(tile)

    show_img("crop_grid",img_copy)

In [15]:
crop_grid("images/dog.jfif",5,5,(0,255,0))

In [26]:
#Q3

img = cv.imread("images/lena.jfif")
img2 = cv.imread("images/coins.jfif")

h, w = img.shape[:2]
img2 = cv.resize(img2, (w, h))


# Perform image blending with smooth transition and display the sequence
for alpha in range(60):
    # Calculate the alpha value for blending (ranges from 0 to 1)
    alpha = alpha / 60

    res = cv.addWeighted(img, alpha, img2, 1 - alpha, 0)

    # Display the blended image with a delay for smoother transition
    cv.imshow("Image Blending", res)
    cv.waitKey(20)  # Adjust the delay (in milliseconds) between each frame

cv.waitKey(0)
cv.destroyAllWindows()

# Adding watermark

In [ ]:
#Q4
watermark=cv.imread("Picture1.png")

show_img("watermark",watermark)

In [ ]:
watermark.shape

In [ ]:
img=cv.imread("images/travel_hd.jpg")

img.shape

In [ ]:
#overlay have same dimension with img but in 0
overlay=np.zeros_like(img)

#get height and width
h,w=img.shape[:2]
hW,wW=watermark.shape[:2]

overlay[h-hW-15:h-15,15:15+wW]=watermark

In [ ]:
#add watermark into the img
watermarked_img=cv.addWeighted(img,1,overlay,0.4,0)

show_img("watermark",watermarked_img,adjust=True)